In [2]:
# Packages

library(tidyverse)

library(emmeans)
library(easystats)

library(arrow)
library(data.table)
library(dtplyr)

library(ggplot2)
library(ggthemes)
library(patchwork)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   4.0.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

# Attaching packages: easystats 0.7.5
✔ bayestestR  0.17.0   ✔ correlation 0.8.8 
✔ datawizard  1.3.0    ✔ effectsize  1.0.1 
✔ insight     1.4.4    ✔ modelbased  0.13.1
✔ performance 0.15.3   ✔ parameters  0.28.3
✔ report      0.6.2    ✔ see         0.12.0



Attaching package: 'arrow'


The following object is masked from 'package:lubridate':

    duration



In [3]:
# Reading the data from parquets; see end of file for original read-in code. 

# df_full <- read_parquet("Data/Full Data.parquet")
df_sent <- read_parquet("Data/Filtered Sentences.parquet")
df_nps <- read_parquet("Data/NPs Only.parquet")

In [4]:
model <- glm(data=df_nps,
                formula = surprisal ~ argPos * definiteness * np_idx,
                family = Gamma(link = 'log')
                )

In [6]:
model_dashboard(
  model,
  check_model_args = NULL,
  parameters_args = NULL,
  performance_args = NULL,
  output_file = "easydashboard.html",
  output_dir = getwd(),
  rmd_dir = system.file("templates/easydashboard.Rmd", package = "easystats"),
  quiet = FALSE,
  browse_html = interactive()
)



processing file: easydashboard.Rmd



1/36                   
2/36 [setup]           
3/36                   
4/36 [easydashboard-1] 
5/36                   
6/36 [check-model]     
7/36                   
8/36 [easydashboard-2] 
9/36                   
10/36 [easydashboard-3] 
11/36                   
12/36 [easydashboard-4] 
13/36                   
14/36 [dot-whisker]     
15/36                   
16/36 [easydashboard-5] 
17/36                   
18/36 [easydashboard-6] 
19/36                   
20/36 [easydashboard-7] 
21/36                   
22/36 [expected-values] 
23/36                   
24/36 [easydashboard-8] 
25/36                   
26/36 [easydashboard-9] 
27/36                   
28/36 [easydashboard-10]
29/36                   
30/36 [easydashboard-11]
31/36                   
32/36 [easydashboard-12]
33/36                   
34/36 [easydashboard-13]
35/36                   
36/36 [easydashboard-14]


output file: C:/Users/vmari/OneDrive - University of Delaware - o365/QP2/QP2 Analysis/easydashboard.knit.md




"C:/Users/vmari/AppData/Local/Pandoc/pandoc" +RTS -K512m -RTS "C:\Users\vmari\ONEDRI~1\QP2\QP2ANA~1\EASYDA~1.MD" --to html4 --from markdown+autolink_bare_uris+tex_math_single_backslash --output pandoca7c0603d4462.html --lua-filter "C:\Users\vmari\AppData\Local\R\win-library\4.5\rmarkdown\rmarkdown\lua\pagebreak.lua" --lua-filter "C:\Users\vmari\AppData\Local\R\win-library\4.5\rmarkdown\rmarkdown\lua\latex-div.lua" --lua-filter "C:\Users\vmari\AppData\Local\R\win-library\4.5\rmarkdown\rmarkdown\lua\table-classes.lua" --embed-resources --standalone --variable bs3=TRUE --standalone --section-divs --template "C:\Users\vmari\AppData\Local\R\win-library\4.5\flexdashboard\www\flex_dashboard\default.html" --include-in-header "C:\Users\vmari\AppData\Local\Temp\Rtmpaq8IeQ\rmarkdown-stra7c026bf6a11.html" --variable theme=cosmo --mathjax --variable "mathjax-url=https://mathjax.rstudio.com/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML" --include-in-header "C:\Users\vmari\AppData\Local\Temp\Rtmpaq8


Output created: C:/Users/vmari/OneDrive - University of Delaware - o365/QP2/QP2 Analysis/easydashboard.html



In [5]:
head(df_nps)

bnc_id,sentence_id,s_text,phr_tok,surprisal,argPos,definiteness,np_idx
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>,<dbl>
A00_0079,A00_0079,FACTSHEET BECOMING AN ACET HOME CARE VOLUNTEER,FACTSHEET,5.8942057,sbj,indef,0
A00_0079,A00_0079,FACTSHEET BECOMING AN ACET HOME CARE VOLUNTEER,AN ACET HOME CARE VOLUNTEER,4.7950663,obj,indef,2
A00_0178,A00_0176,Tear Fund Support Ugandan Work,Tear Fund,8.3891602,sbj,def,0
A00_0178,A00_0176,Tear Fund Support Ugandan Work,Ugandan Work,2.9604492,obj,indef,3
A01_0034,A01_0034,A new germ enters the body .,A new germ,4.9772135,sbj,indef,0
A01_0034,A01_0034,A new germ enters the body .,the body,0.5557861,obj,def,4


In [21]:
context <- function(df, target_id, num=3) {

    dt <- as.data.table(df)

    u_ids <- unique(dt$sentence_id)
    

    target_pos <- match(target_id, u_ids)
    
    if (is.na(target_pos)) return(NULL)
    

    start_idx <- max(1, target_pos - num)
    end_idx   <- target_pos - 1
    
    prev_ids <- if (target_pos > 1) u_ids[start_idx:end_idx] else integer(0)
    
    dt_context <- dt[.(prev_ids), on = "sentence_id", mult = "first"]
    dt_context[, phr_tok := ""]
    dt_context[, surprisal := NA]
    
    dt_target <- dt[sentence_id == target_id]
    
    result <- rbind(dt_context, dt_target)

    result <- result %>%  select(bnc_id, sentence_id, s_text, phr_tok, surprisal, argPos, definiteness)

    # result <- result %>% select(-grp_idx)

    as_tibble(result)
    }

    context_full <- function(df, target_id, num=3) {
    dt <- as.data.table(df)
    dt[, grp_idx := rleid(sentence_id)]
    target_grp <- dt[sentence_id == target_id, grp_idx[1]]
    result <- dt[grp_idx >= (target_grp - num) & grp_idx <= target_grp]
    result <- result %>% select(-grp_idx) %>%  select(bnc_id, sentence_id, s_text, phr_tok, surprisal, argPos, definiteness)
    as_tibble(result)
}

In [5]:
# df_nps %>% 
#     filter() %>% 
#     select(Sentence_Text, Phrase_Token, argPos, surprisal, Sentence_ID) %>%
#     group_by(Sentence_ID)%>%
#     mutate(max_surprisal = max(surprisal)) %>% 
#     ungroup() %>% 
#     arrange(max_surprisal)%>%
#     head(20)

In [37]:


df_nps %>% 
    select(s_text, phr_tok, np_idx, argPos, definiteness, surprisal, sentence_id, bnc_id)%>%
    filter(definiteness == "def")%>%
    arrange(surprisal) %>%
    head(837)

s_text,phr_tok,np_idx,argPos,definiteness,surprisal,sentence_id,bnc_id
<chr>,<chr>,<dbl>,<fct>,<fct>,<dbl>,<chr>,<chr>
Sofia 's eyes met Caroline 's .,Caroline,4,obj,def,0.0004274845,JY7_4504,JY7_4504
Clyde eyed Martin suspiciously .,Martin,2,obj,def,0.0009312630,FRS_3322,FRS_3322
Preston studied him curiously .,him,2,obj,def,0.0026130676,F9C_2025,F9C_2026
Ukrainians to close Chernobyl,Chernobyl,3,obj,def,0.0034236908,J31_0352,J31_0352
Rogers studied him curiously .,him,2,obj,def,0.0038032532,HRA_0977,HRA_0977
Preston regarded her uneasily .,her,2,obj,def,0.0046005249,F9C_0876,F9C_0877
Pendulum videos support WWF,WWF,3,obj,def,0.0052490234,HJM_0149,HJM_0149
3 Man disobeys God,God,3,obj,def,0.0064163208,EFT_0744,EFT_0744
Pearce backs Clough,Clough,2,obj,def,0.0078735352,CH3_0928,CH3_0929


In [19]:
context(df_full, "GUL_0694")

bnc_id,sentence_id,s_text,phr_tok,surprisal,argPos,definiteness
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<fct>
GUL_0691,GUL_0691,‘ I 've got my car here . ’,,NA,NA,NA
GUL_0692,GUL_0692,It was dark when Karen left the factory .,,NA,sbj,def
GUL_0693,GUL_0693,"She smiled at all her friends , got into her small white car and drove",,NA,sbj,def
GUL_0694,GUL_0694,Nobody ever found the brown envelope,Nobody,19.703125,sbj,def
GUL_0694,GUL_0694,Nobody ever found the brown envelope,ever,4.441406,NA,NA
GUL_0694,GUL_0694,Nobody ever found the brown envelope,found,5.121094,NA,NA
GUL_0694,GUL_0694,Nobody ever found the brown envelope,the brown envelope,4.322266,obj,def
GUL_0694,GUL_0694,Nobody ever found the brown envelope,the brown envelope,4.322266,obj,def
GUL_0694,GUL_0694,Nobody ever found the brown envelope,the brown envelope,4.322266,obj,def
